In [1]:
from requests_html import HTMLSession
import time
import re
import pandas as pd
import csv
import datetime

In [2]:
session = HTMLSession()
content_page_url = 'http://www.passiontimes.hk/category/1/1/?page={}'
content_url = 'http://www.passiontimes.hk/article/08-31-2018/{}'

### Get content page num

In [3]:
# def get_page_num():
#     start_content_page = content_page_url.format(1)
#     r = session.get(start_content_page)
#     page_num = int(r.html.find('h5>span')[0].text.split('頁')[0].split('共')[1])
#     assert type(page_num) == int
#     return page_num

### Extract page id from single content page

In [4]:
def get_ids(page_num):
    r = session.get(content_page_url.format(page_num))
    url_list = [tag.attrs['href'] for tag in r.html.find('div.textBox>h4>a[href*="www.passiontimes.hk"]')]
    return list(set(url_list))

### Content selection

In [5]:
def get_single_content(url):
    dict_return = {}
    r = session.get(url)
    page_id = url.split('/')[-1]
    dict_return['title'] = r.html.find('header>h1')[0].text
    dict_return['category'] = r.html.find('div.page-path>h4>a')[-1].text
    dict_return['content'] = ''.join([tag.text for tag in r.html.find('div.article-body>div,div.article-body>p')])
    date = r.html.find('time.published')[0].attrs['datetime']
    date = datetime.datetime.strptime(date,"%m-%d-%Y")
    dict_return['date'] = date
    dict_return['url'] = url
    return dict_return

### Multi Pages of a content page

In [6]:
def multi_content(list_urls):
    for url in list_urls:
        yield get_single_content(url)

### csv writer functions

In [7]:
fieldnames = ['title', 'category', 'content', 'date', 'url']
def writer_initiator():
    with open('scraper.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

In [8]:
def writer(multi_content):
    with open('scraper.csv','a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        while True:
            try:
                writer.writerow(next(multi_content))
            except StopIteration:
                break

### Scrapper

In [9]:
current_page = 1
stop_page = 15 #total 680

In [10]:
writer_initiator()
while True:
    print('Scrapping Content Page {}.'.format(current_page), end="\r")
    list_urls = get_ids(current_page)
    articles_generator = multi_content(list_urls)
    writer(articles_generator)
    current_page += 1
    if current_page == stop_page+1:
        break